In [18]:
import random
import gurobipy as gp
from gurobipy import GRB
import copy
import numpy as np

n=10
def generate_data(n):
    # Initializing an empty adjacency matrix with zeros
    W = [[0] * n for _ in range(n)]

    # Generate random edge weights (assuming non-negative weights)
    for i in range(n):
        for j in range(i+1, n):
            t = random.randint(0,2)
            if t == 0:
                weight = 0
            else:
                weight = random.randint(1,10)
            # weight = random.uniform(0, 2)  # Adjust the range as needed
            # Ensure that edges are undirected by setting W[i][j] = W[j][i] = weight
            W[i][j] = W[j][i] = weight
    return W

In [19]:
def BP(W,n):
    # Create a new model
    m = gp.Model("mip1")
    # Create variable

    x = {}
    x = m.addVars(range(n),vtype = "B", name="x")
    y = m.addVars(range(n),range(n),vtype = "B", name="y")

    m.setObjective(sum(W[i][j]*y[i,j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)
    m.addConstrs(y[i,j]<=(x[i]+x[j]) for i in range(n) for j in range(n))
    m.addConstrs(y[i,j]+x[i]+x[j]<=2 for i in range(n) for j in range(n))
    m.optimize()
    y_result=[]

    for v in x.values():
        if v.X == 0 or v.X==-0:
            y_result.append(0)
        else:
            y_result.append(1)
    return y_result

In [20]:
def count_weight(W:list, A:list, B:list):
    total_weight = 0
    for i in A:
        for j in B:
            total_weight += W[i][j]
    return total_weight

In [21]:
def weight_change(W, A, B, current_weight):
    weight_change_list = []
    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        if i in A:

            temp_B.append(i)
            temp_A.remove(i)
        else:

            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        result = new_weight - current_weight
        weight_change_list.append(result)
    return weight_change_list

In [22]:
def random_set():
    A = []
    B = []
    xdata_gh = []

    for i in range(n):
        r = random.randint(0,1)
        if r == 0:
            xdata_gh.append(0)
            A.append(i)
        else:
            xdata_gh.append(1)
            B.append(i)
    return xdata_gh, A, B

In [23]:
def GH(W, A, B, n):
    total_weight = count_weight(W, A, B)

    for i in range(n):
        temp_A = A.copy()
        temp_B = B.copy()
        if i in A:
            temp_B.append(i)
            temp_A.remove(i)
        else:
            temp_A.append(i)
            temp_B.remove(i)
        new_weight = count_weight(W, temp_A, temp_B)
        if new_weight >= total_weight:
            A = temp_A
            B = temp_B
            total_weight = new_weight
    A = sorted(A)
    B = sorted(B)
    total_weight = count_weight(W, A, B)
    return total_weight, A, B

In [24]:
# k=5

# A_GH_list = []
# B_GH_list = []
# gh_weight_list = []
# raw_data = []
# xdata1_l = []
# ydata1_l = []

# for i in range(k):
#     sum_W = []
#     data_list = []
#     data = generate_data(n)

#     xdata_gh, A, B = random_set()
#     gh_weight, A_GH, B_GH = GH(data,A,B,n)

#     A_GH_list.append(A_GH)
#     B_GH_list.append(B_GH)
#     gh_weight_list.append(gh_weight)

#     current_weight = count_weight(data,A,B)
#     weight_diff = weight_change(data,A,B,current_weight)

#     # result = BP(data, n)
    # ydata1_l.append(result)
    
    # for j in data:
    #     sum_W.append(sum(j))

    # for j in range(len(sum_W)):
    #     test = []
    #     test.append(sum_W[j])
    #     test.append(xdata_gh[j])
    #     test.append(weight_diff[j])
    #     xdata1_l.append(test)
    # raw_data.append(data)

In [25]:
def get_data(k):
    A_GH_list = []
    B_GH_list = []
    gh_weight_list = []
    raw_data = []
    xdata1_l = []
    ydata1_l = []

    for i in range(k):
        sum_W = []
        data_list = []
        data = generate_data(n)

        xdata_gh, A, B = random_set()
        gh_weight, A_GH, B_GH = GH(data,A,B,n)

        A_GH_list.append(A_GH)
        B_GH_list.append(B_GH)
        gh_weight_list.append(gh_weight)

        current_weight = count_weight(data,A,B)
        weight_diff = weight_change(data,A,B,current_weight)

        result = BP(data, n)
        for j in range(len(result)):
            ydata1_l.append(result[j])
        
        for j in data:
            sum_W.append(sum(j))
        # data_list.append(sum_W)
        # data_list.append(xdata_gh)
        # data_list.append(weight_diff)
        # xdata1_l.append(data_list)

        for j in range(len(sum_W)):
            test = []
            test.append(sum_W[j])
            test.append(xdata_gh[j])
            test.append(weight_diff[j])
            xdata1_l.append(test)

    return xdata1_l, ydata1_l, A_GH_list, B_GH_list, gh_weight_list, raw_data

In [26]:
k1 = 5
xtrain_list, ytrain_list, A_GH_train, B_GH_train, gh_weight_train, raw_data = get_data(k1)

k2 = 1
xtest_list, ytest_list, A_GH_test, B_GH_test, gh_weight_test, raw_data = get_data(k2)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 200 rows, 110 columns and 580 nonzeros
Model fingerprint: 0x19b3c38a
Variable types: 0 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 136 rows and 68 columns
Presolve time: 0.00s
Presolved: 64 rows, 42 columns, 192 nonzeros
Variable types: 0 continuous, 42 integer (42 binary)
Found heuristic solution: objective 226.0000000

Root relaxation: objective 3.760000e+02, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  

In [27]:
# A_BP = []
# B_BP = []

# for i in ytest_list:
#     temp_A = []
#     temp_B = []
#     for j in range(len(i)):
#         if i[j]>0.5:
#             temp_A.append(j)
#         else:
#             temp_B.append(j)
#     A_BP.append(temp_A)
#     B_BP.append(temp_B)

In [28]:
# BP_result = 0
# GH_result = 0
# for i in range(len(A_BP)):
#     GH_weight = count_weight(raw_data[i], A_GH_test[i], B_GH_test[i])
#     BP_weight = count_weight(raw_data[i], A_BP[i], B_BP[i])
#     if BP_weight >= GH_weight:
#         BP_result += 1
#     else:
#         GH_result += 1

# print(f"Outcome of the BP where they have a higher performance is {BP_result/len(A_BP)}%")
# print(f"Outcome of the GH where they have a higher performance is {GH_result/len(A_BP)}%")

In [29]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

def build_binary_model(xtrain, ytrain, xtest, ytest, nrlayers, nrnodes, rate, k):
    # This function builds the model with nrlayes layers and nrnodes nodes in each layer
    # nrlayers is a number and nrnodes is a vector specifying how many nodes each layer should contain
    model = keras.Sequential()
    numclasses = k

    # add the layers
    for i in range(nrlayers + 1):
        if (i == 0):
            # input layer
            model.add(keras.Input(shape=input_shape))
        else:
            # hidden layers
            nrnode = nrnodes[i - 1]
            model.add(keras.layers.Dense(nrnode, activation="sigmoid"))

    model.add(keras.layers.Dense(numclasses, activation="sigmoid"))

    # give summary of model
    model.summary()

    # train model
    batch_size = 1 # Based on SGD --> 1 gradient is calculated using 32 data points every time
    #so in each epoch, training_size/batch_size many gradient calculations and weight updates
    epochs = 5 # How many times to go through the data to complete the SGD

    opt = keras.optimizers.SGD(learning_rate=rate)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])
    
    model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epochs, validation_split=0.1,)
    score_train = model.evaluate(xtrain, ytrain, verbose=1)
    print(f"Train loss:, {score_train[0]}, Train accuracy:, {score_train[1]}")
    # get results
    score = model.evaluate(xtest, ytest, verbose=1)
    print(f"Test loss: {score[0]}, Test accuracy:, {score[1]}")
    return model

In [30]:
xtrain = np.array(xtrain_list)
xtest = np.array(xtest_list)
ytrain = np.array(ytrain_list)
ytest = np.array(ytest_list)

In [31]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(50, 3)
(10, 3)
(50,)
(10,)


In [32]:
ytrain

array([1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0])

In [35]:
# xtrain=np.reshape(xtrain,[50, 3])
# xtest=np.reshape(xtest,[30, 3])

input_shape = (3,)
models1 = build_binary_model(xtrain, ytrain, xtest, ytest, 2, [2*n,1.75*n], 0.3, 1)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 20)                80        
                                                                 
 dense_10 (Dense)            (None, 17)                357       
                                                                 
 dense_11 (Dense)            (None, 1)                 18        
                                                                 
Total params: 455 (1.78 KB)
Trainable params: 455 (1.78 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
45/45 [==============================] - 1s 8ms/step - loss: 0.8222 - accuracy: 0.4667 - val_loss: 0.5274 - val_accuracy: 0.8000
Epoch 2/5
45/45 [==============================] - 0s 3ms/step - loss: 0.6973 - accuracy: 0.6889 - val_loss: 0.9731 - val_accuracy: 0.2000
Epoch 3/5


In [39]:
models1.predict(xtest)

1/1 [==============================] - 0s 24ms/step


array([[0.7235045 ],
       [0.73561656],
       [0.7339592 ],
       [0.73050517],
       [0.7266104 ],
       [0.727315  ],
       [0.7359267 ],
       [0.7337134 ],
       [0.73355645],
       [0.7359089 ]], dtype=float32)

In [37]:
ytest

array([0, 1, 1, 0, 1, 1, 0, 1, 0, 0])

In [ ]:
NN = models1.predict(xtest)
NN_ytest = NN.tolist()
NN_ytest = [[int(x) for x in sublist] for sublist in NN_ytest]

In [ ]:
A_NN = []
B_NN = []

for i in NN_ytest:
    temp_A = []
    temp_B = []
    for j in range(len(i)):
        if i[j]>0.5:
            temp_A.append(j)
        else:
            temp_B.append(j)
    A_NN.append(temp_A)
    B_NN.append(temp_B)

In [ ]:
NN_result = 0
GH_result = 0
for i in range(len(A_NN)):
    GH_weight = count_weight(raw_data[i], A_GH_test[i], B_GH_test[i])
    NN_weight = count_weight(raw_data[i], A_NN[i], B_NN[i])

    if NN_weight >= GH_weight:
        NN_result += 1
    else:
        GH_result += 1

print(f"Outcome of the MLGH where they have a higher performance is {NN_result/len(A_NN)}%")
print(f"Outcome of the GH where they have a higher performance is {GH_result/len(A_NN)}%")